# Multi-View-Majority-Vote-Learning-Algorithms-Direct-Minimization-of-PAC-Bayesian-Bounds

In [1]:
import numpy as np
from sklearn.utils import check_random_state
from sklearn.preprocessing import MinMaxScaler
RAND = check_random_state(42)
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn.parameter import Parameter
from collections import OrderedDict
import torch.nn.functional as F
from torch.utils.data import DataLoader,Dataset
from mvlearn.datasets import load_UCImultifeature
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


from dNDF import MultiViewBoundsDeepNeuralDecisionForests


from mvb import RandomForestClassifier as RFC


# Multi-view dataset : Nhanes 4 views https://wwwn.cdc.gov/nchs/nhanes/continuousnhanes/overview.aspx?BeginYear=2017
from data.datasets import (Nhanes, 
                           train_test_split,
                           train_test_merge)

In [2]:
mv_data, labels = load_UCImultifeature()
np.unique(labels)

array([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.])

In [3]:
scaler = MinMaxScaler()
for i in range(len(mv_data)):
    mv_data[i] = scaler.fit_transform(mv_data[i])

In [4]:
mvb = MultiViewBoundsDeepNeuralDecisionForests(nb_estimators=3, nb_views=len(mv_data), depth =3,used_feature_rate=0.8)

In [5]:
mvb.fit(mv_data,labels)

MultiViewBoundsDeepNeuralDecisionForests(depth=3, nb_estimators=3, nb_views=6,
                                         posterior=array([0.33333333, 0.33333333, 0.33333333]),
                                         used_feature_rate=0.8)

In [6]:
posterior_Qv , posterior_rho = mvb.optimize_rho('Lambda',delta=0.05, eps=10**-9)

[[0.43455497 0.53548387 0.42140921]
 [0.42607527 0.5114094  0.45643154]
 [0.46986301 0.64058355 0.6489071 ]
 [0.24217687 0.36118598 0.24492558]
 [0.56381487 0.54143646 0.47936085]
 [0.4986631  0.55033557 0.45121951]]
Parameter containing:
tensor([0.3333, 0.3333, 0.3333], requires_grad=True)
Parameter containing:
tensor([0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667], requires_grad=True)
Avant normalisation et clampage: Parameter containing:
tensor([0.3333, 0.3333, 0.3333], requires_grad=True)
Après normalisation et clampage: Parameter containing:
tensor([0.3333, 0.3333, 0.3333], requires_grad=True)
Avant normalisation et clampage: Parameter containing:
tensor([0.3333, 0.3333, 0.3333], requires_grad=True)
Après normalisation et clampage: Parameter containing:
tensor([0.3333, 0.3333, 0.3333], requires_grad=True)
Avant normalisation et clampage: Parameter containing:
tensor([0.3333, 0.3333, 0.3333], requires_grad=True)
Après normalisation et clampage: Parameter containing:
tensor([0.3333

In [7]:
print(posterior_Qv[0])
print(posterior_Qv[1])
print(posterior_Qv[2])
print(posterior_Qv[3])
print(posterior_Qv[4])
print(posterior_Qv[5])

Parameter containing:
tensor([0.3379, 0.3222, 0.3399], requires_grad=True)
Parameter containing:
tensor([0.3393, 0.3262, 0.3346], requires_grad=True)
Parameter containing:
tensor([0.3469, 0.3271, 0.3261], requires_grad=True)
Parameter containing:
tensor([0.3514, 0.2985, 0.3501], requires_grad=True)
Parameter containing:
tensor([0.3282, 0.3314, 0.3404], requires_grad=True)
Parameter containing:
tensor([0.3335, 0.3268, 0.3397], requires_grad=True)


In [8]:
posterior_Qv = [F.softmax(param, dim=0) for param in posterior_Qv]
posterior_Qv

[tensor([0.3348, 0.3296, 0.3355], grad_fn=<SoftmaxBackward0>),
 tensor([0.3353, 0.3309, 0.3337], grad_fn=<SoftmaxBackward0>),
 tensor([0.3379, 0.3312, 0.3309], grad_fn=<SoftmaxBackward0>),
 tensor([0.3393, 0.3218, 0.3389], grad_fn=<SoftmaxBackward0>),
 tensor([0.3316, 0.3327, 0.3357], grad_fn=<SoftmaxBackward0>),
 tensor([0.3334, 0.3312, 0.3355], grad_fn=<SoftmaxBackward0>)]

In [9]:
posterior_rho

Parameter containing:
tensor([3.9947e-02, 3.5136e-02, 6.9151e-04, 9.2198e-01, 9.1159e-04, 1.3379e-03],
       requires_grad=True)